In [426]:
from pymystem3 import Mystem

!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

import gensim
from pymystem3 import Mystem

stemmer = Mystem()
lemmas = stemmer.lemmatize("Красивая мама красиво мыла раму")
print(lemmas)

--2021-10-18 23:29:48--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.241, 5.45.205.242, 5.45.205.243, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.241|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-man01i.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2021-10-18 23:29:49--  http://cache-man01i.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-man01i.cdn.yandex.net (cache-man01i.cdn.yandex.net)... 5.45.205.221, 2a02:6b8::3:221
Connecting to cache-man01i.cdn.yandex.net (cache-man01i.cdn.yandex.net)|5.45.205.221|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.19’

mystem-3.0-linux3.1 100%[===================>]  15.70M

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_PATH = "/content/drive/MyDrive/YandexCup/"


Mounted at /content/drive


In [ ]:
!pip install -q transformers
!pip install -q natasha
!pip install -q pymorphy2
!pip install -q -U pymorphy2-dicts-ru
!pip install -q hnswlib
!pip install -q pymystem3
!pip install -q pymorphy2
!pip install -q -U pymorphy2-dicts-ru
!pip install https://github.com/kpu/kenlm/archive/master.zip
!pip install -q kenlm


     / 540 kB 2.2 MB/s
  Created wheel for kenlm: filename=kenlm-0.0.0-cp37-cp37m-linux_x86_64.whl size=2334453 sha256=b69cae99c61d29b32875d699bf5c2b26a210f3824e72f8167aa2c2597fb8fa3f
  Stored in directory: /tmp/pip-ephem-wheel-cache-5p5vpi35/wheels/3d/aa/02/7b4a2eab5d7a2a9391bd9680dbad6270808a147bc3b7047e4e
Successfully built kenlm


#SETUP:

In [ ]:
import numpy as np

import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch import softmax, sigmoid
from tqdm.auto import tqdm
import argparse
from pymystem3 import Mystem
import sys
from collections import Counter
from functools import partial
import kenlm

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/YandexCup/trained_roberta")

model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/YandexCup/trained_roberta").cuda()

lm = kenlm.Model('/content/drive/MyDrive/YandexCup/lm.binary')



TOXIC_CLASS=-1
TOKENIZATION_TYPE='sentencepiece'


ALLOWED_ALPHABET=list(map(chr, range(ord('а'), ord('я') + 1)))
ALLOWED_ALPHABET.extend(map(chr, range(ord('a'), ord('z') + 1)))
ALLOWED_ALPHABET.extend(list(map(str.upper, ALLOWED_ALPHABET)))
ALLOWED_ALPHABET = set(ALLOWED_ALPHABET)



def logits_to_toxic_probas(logits):
    if logits.shape[-1] > 1:
        activation = lambda x: softmax(x, -1)
    else:
        activation = sigmoid
    return activation(logits)[:, TOXIC_CLASS].cpu().detach().numpy()


def is_word_start(token):
    if TOKENIZATION_TYPE == 'sentencepiece':
        return token.startswith('▁')
    if TOKENIZATION_TYPE == 'bert':
        return not token.startswith('##')
    raise ValueError("Unknown tokenization type")


def normalize(sentence, max_tokens_per_word=20):
    def validate_char(c):
        return c in ALLOWED_ALPHABET
    
    sentence = ''.join(map(lambda c: c if validate_char(c) else ' ', sentence.lower()))
    ids = tokenizer(sentence)['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(ids)[1:-1]
    
    result = []
    num_continuation_tokens = 0
    for token in tokens:
        if not is_word_start(token):
            num_continuation_tokens += 1
            if num_continuation_tokens < max_tokens_per_word:
                result.append(token.lstrip('#▁'))
        else:
            num_continuation_tokens = 0
            result.extend([' ', token.lstrip('▁#')])
    
    return ''.join(result).strip()


def iterate_batches(data, batch_size=40):
    batch = []
    for x in data:
        batch.append(x)
        if len(batch) >= batch_size:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch


def predict_toxicity(sentences, batch_size=5, threshold=0.5, return_scores=False, verbose=True, device='cuda'):
    results = []
    for batch in iterate_batches(sentences, batch_size):
        normlized = [normalize(sent, max_tokens_per_word=5) for sent in batch]
        tokenized = tokenizer(normlized, return_tensors='pt', padding=True, max_length=512, truncation=True)
    
        logits = model.to(device)(**{key: val.to(device) for key, val in tokenized.items()}).logits
        preds = logits_to_toxic_probas(logits)
        if not return_scores:
            preds = preds >= threshold
        results.extend(preds)
    return results

#EMBEDDINGS:

In [ ]:
def get_w2v_indicies(a):
    res = []
    if isinstance(a, str):
        a = a.split()
    for w in a:
        if w in embs_voc:
            res.append((w, embs_voc[w]))
        else:
            
            for lemma in stemmer.lemmatize(w):
                if lemma.isalpha():
                    res.append((w, embs_voc.get(lemma, None)))
    return res


def calc_embs(words):
    words = ' '.join(map(normalize, words))
    inds = get_w2v_indicies(words)
    return [(w, i if i is None else embs_vectors[i]) for w, i in inds]


def calc_single_embedding_dist(word1, word2):
    if (word1 in COMMENTS_VOCAB) and (word2 in COMMENTS_VOCAB[word1]):
        
        return COMMENTS_VOCAB[word1][word2]
    if (word2 in COMMENTS_VOCAB) and (word1 in COMMENTS_VOCAB[word2]):
        
        return COMMENTS_VOCAB[word2][word1]

    a = calc_embs([word1])[0]
    b = calc_embs([word2])[0]



    a_s, a_v = a  #слово, вектор
    b_s, b_v = b  #слово, вектор
    if a_s == b_s: #если слова равны
        COMMENTS_VOCAB[word1][word2] = 0.0
        COMMENTS_VOCAB[word2][word1] = 0.0
        return 0.0
    if a_v is None or b_v is None: #если одно из векторов нет - дать пизды 
        COMMENTS_VOCAB[word1][word2] = 1.0
        COMMENTS_VOCAB[word2][word1] = 1.0
        return 1.0
    a = a_v
    b = b_v
    # inexact match is punished by 0.1
    result = 0.1 + 0.9 * (1 - a.dot(b) / np.linalg.norm(a) / np.linalg.norm(b)) / 2

    COMMENTS_VOCAB[word1][word2] = result
    COMMENTS_VOCAB[word2][word1] = result

    return result


def greedy_match_embs(a, b, max_dist=99999, cache=None, a_ind=0, b_ind=0):
    a_len = len(a) - a_ind #сколько слов мы еще не рассмотрели в а
    b_len = len(b) - b_ind #сколько слов мы еще не рассмотрели в b
    minlen = min(a_len, b_len) #минимальная длина строки
    maxlen = max(a_len, b_len) #минимальная длина строки
    if minlen == 0: 
        return np.minimum(maxlen, max_dist) 
    if maxlen - minlen >= max_dist: 
        return max_dist 
    
    if cache is None:
        cache = {}
    
    cache_key = (a_len, b_len)
    if cache_key in cache:
        return cache[cache_key]
        
    min_dist = max_dist
    first_dist = calc_single_embedding_dist(a[a_ind], b[b_ind])
    if max_dist >= first_dist:
        min_dist = np.minimum(min_dist, first_dist + greedy_match_embs(
            a, b, max_dist, cache, a_ind + 1, b_ind + 1
        ))
    
    if first_dist > 0 and max_dist >= 1:
        min_dist = np.minimum(min_dist, 1 + greedy_match_embs(
            a, b, max_dist, cache, a_ind + 1, b_ind
        ))
        min_dist = np.minimum(min_dist, 1 + greedy_match_embs(
            a, b, max_dist, cache, a_ind, b_ind + 1
        ))
    
    cache[cache_key] = min_dist
    
    return min_dist





def distance_score(original, fixed):
    original = original.split()
    fixed = fixed.split()

    
    clip_distance = 5  # this clips long computations
    
    numerator = greedy_match_embs(original, fixed, max_dist=clip_distance)

    denominator = 0.6 * np.log(1 + len(original))

    return np.exp(-(numerator / denominator) ** 2)

In [ ]:
def compute_lmdiff(original, fixed):
    original_lm_logproba = lm.score(original, bos=True, eos=True)
    fixed_lm_logproba = lm.score(fixed, bos=True, eos=True)
    
    probability_fraction = 10**((fixed_lm_logproba - original_lm_logproba) / 25)
    
    return np.clip(probability_fraction, 0.0, 1.0)
    

def compute_score(original_sentences, fixed_sentences, threshold=0.5, batch_size=5, show_res = False, return_result = False):
    fixed_toxicities = predict_toxicity(fixed_sentences, threshold=threshold, batch_size=batch_size, return_scores=True)
    scores = []
    lmdiffs = []
    emb_dists = []
    for original_sentence, fixed_sentence, fixed_toxicity in tqdm(zip(
        original_sentences, fixed_sentences, fixed_toxicities
    ), miniters=250, disable= True):
        original_sentence = normalize(original_sentence)
        fixed_sentence = normalize(fixed_sentence)
        
        distance = distance_score(original_sentence, fixed_sentence)
        lmdiff = compute_lmdiff(original_sentence, fixed_sentence)
        
        score = (1 - fixed_toxicity) * distance * lmdiff
        
        lmdiffs.append(lmdiff)
        emb_dists.append(distance)
        scores.append(score)
        
    if show_res:
        print('average toxicity:', np.mean(fixed_toxicities), file=sys.stderr)
        print('mean lmdiff:', np.mean(lmdiffs), file=sys.stderr)
        print('mean distance_score:', np.mean(emb_dists), file=sys.stderr)
        print('score', np.mean(scores) * 100, file=sys.stderr)
        
    if return_result:
        return {"scores" : scores, 
                "toxicities" : fixed_toxicities, 
                "lmdiffs" : lmdiffs,
                "distance_scores" : emb_dists}
    else:
        return scores

def new_checker(original_texts, fixed_texts, device='cuda', show_res = False, return_result = False):
    original_texts = list(map(str.strip, original_texts))
    fixed_texts = list(map(str.strip, fixed_texts))
    
    assert len(original_texts) == len(fixed_texts)
    
    with torch.inference_mode(True):
        if return_result:
            return compute_score(original_texts, fixed_texts, show_res = show_res, return_result=return_result)
        return compute_score(original_texts, fixed_texts, show_res = show_res)


In [ ]:
embs_file = np.load(DRIVE_PATH + 'embeddings_with_lemmas.npz', allow_pickle=True)
embs_vectors = embs_file['vectors']
embs_voc = embs_file['voc'].item()


embs_voc_normalized = set([normalize(word) for word in tqdm(embs_voc.keys())])
print(len(embs_voc_normalized))

  0%|          | 0/162690 [00:00<?, ?it/s]

148576


In [ ]:
texts = []
with open(DRIVE_PATH + 'public_testset.txt', 'rt') as f:
    for line in f:
        texts.append(normalize(line))

comments_voc = set()
for text in texts:
    comments_voc.update(text.split())
print(len(comments_voc))

22005


#Обучение супер словаря и toxicity 

In [ ]:
EMBEDDING_VOCAB = embs_voc_normalized | comments_voc
print(len(EMBEDDING_VOCAB))

163532


In [ ]:
def improve_toxicity(data):

    words = set()
    for text in data:
        words.update(text.split())
    words = sorted(words)

    with torch.inference_mode():
        word_toxicities = predict_toxicity(words, batch_size=100, return_scores=True)

    before = len(toxicity)
    for word, tox in zip(words, word_toxicities):
        toxicity[word] = tox
    print("New words :", len(toxicity) - before)
    print("Cur len :", len(toxicity))

In [ ]:
#Уменьшаем EMBEDDING_VOCAB в зависимости от токсичности
toxicity = dict()
improve_toxicity(EMBEDDING_VOCAB)

In [383]:
NONTOXIC_EMBEDDING_VOCAB = set()

In [ ]:
#ТОП НЕТОКСИЧНЫХ ИЗ ЭМБЕДДИНГОВ
NONTOXIC_EMBEDDING_VOCAB .update({word for word, tox in toxicity.items() if tox < 0.0098})
print(len(NONTOXIC_EMBEDDING_VOCAB))

In [414]:
#РУЧКАМИ ПОДОБРВАЛИ
SPECIAL_WORDS = ["спасибо", "любовь", "он", 'cardboard', 'замечательную', 'software', 'workbook', "спасибочки", "textbook"]
for word in SPECIAL_WORDS:
    NONTOXIC_EMBEDDING_VOCAB.add(word)
print(len(NONTOXIC_EMBEDDING_VOCAB))

153


In [381]:
#ТОП РУССКИХ
top_russian = set()
with open(DRIVE_PATH + 'dicitionary/top_russian.txt', 'rt') as f:
    for line in f:
        w = normalize(line)
        if w and w in embs_voc_normalized:
            top_russian.add(normalize(line))

word_toxicities = predict_toxicity(top_russian, batch_size=100, return_scores=True)
top_russian = set(word for word, tox in zip(top_russian, word_toxicities) if tox < 0.05)

NONTOXIC_EMBEDDING_VOCAB.update(top_russian)
print(len(NONTOXIC_EMBEDDING_VOCAB))

245
248
628


In [401]:
#СТОП-СЛОВА
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords


russian_stopwords = []
for word in stopwords.words("russian"):
    w = normalize(word)
    if w and w in embs_voc_normalized:
        russian_stopwords.append(w)


print(len(russian_stopwords))
NONTOXIC_EMBEDDING_VOCAB.update(set(russian_stopwords))
print(len(NONTOXIC_EMBEDDING_VOCAB))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
145
152


In [391]:
COMMENTS_VOCAB = dict()

In [415]:
#Обучаем COMMENTS_VOCAB
for word in NONTOXIC_EMBEDDING_VOCAB | comments_voc:
    if word not in COMMENTS_VOCAB:
        COMMENTS_VOCAB[word] = {} 
print(len(COMMENTS_VOCAB))


22010


In [399]:
for key in tqdm(NONTOXIC_EMBEDDING_VOCAB):
    for word in comments_voc:
        if word and key:
            COMMENTS_VOCAB[key][word] = calc_single_embedding_dist(key, word)

  0%|          | 0/152 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [417]:
#Дообучаем COMMENTS_VOCAB
dsts = []
for text in tqdm(texts):
    dsts.append(distance_score(text, text))

print(sum(dsts)/len(dsts))

  0%|          | 0/2500 [00:00<?, ?it/s]

1.0


In [402]:
#СОХРАНИТЬ
import pickle as pkl

with open(DRIVE_PATH + 'EMBS.pkl', 'wb') as f:
    pkl.dump(COMMENTS_VOCAB, f)

In [418]:
def get_top_n(word, n):
    candidates = []
    for candidate in NONTOXIC_EMBEDDING_VOCAB:
        if word in COMMENTS_VOCAB[candidate]:
            dist = COMMENTS_VOCAB[candidate][word]
            #tox = toxicity[candidate]
            score = (1 - dist) #+ (1 - tox)*1.1
            
            candidates.append([score, candidate])
    candidates.sort(reverse = True)
    #print(candidates[:n])
    return [word for score, word in candidates[:n]]

get_top_n("любовь", 30)

['любовь',
 'ты',
 'ведь',
 'никогда',
 'себе',
 'всегда',
 'замечательную',
 'тебя',
 'потому',
 'конечно',
 'мой',
 'она',
 'я',
 'мы',
 'разве',
 'сам',
 'ней',
 'кто',
 'спасибочки',
 'вам',
 'всех',
 'все',
 'тут',
 'тоже',
 'да',
 'тогда',
 'он',
 'но',
 'уж',
 'спасибо']

#ИЛИ Загрузка этого словаря

In [ ]:
import pickle as pkl

with open(DRIVE_PATH + 'EMBS.pkl', 'rb') as f:
    COMMENTS_VOCAB = pkl.load(f)

In [ ]:
def sort_by_toxicity(words):
    toxicities = [toxicity.get(w) if w in toxicity else predict_toxicity([w], return_scores = True, off_tqdm=True) for w in words]
  
    result = [[toxicities[i], i, words[i]] for i in range(len(words))]
    result.sort(reverse = True)
    return result



line = np.random.choice(texts)
pred = predict_toxicity([line])
line_words = normalize(line).split()
print(sort_by_toxicity(line_words))

[[0.85235643, 16, 'идиотом'], [0.5289828, 7, 'банды'], [0.4337692, 6, 'путинской'], [0.29428363, 2, 'выборах'], [0.18188836, 15, 'полным'], [0.15841357, 8, 'но'], [0.15284795, 5, 'кроме'], [0.14809723, 3, 'победят'], [0.13659135, 1, 'честных'], [0.1308763, 4, 'все'], [0.12726091, 9, 'их'], [0.12318689, 17, 'чтобы'], [0.12005733, 20, 'понимать'], [0.11746098, 19, 'не'], [0.11746098, 10, 'не'], [0.112177566, 0, 'в'], [0.09537505, 11, 'будет'], [0.09521881, 13, 'надо'], [0.060944885, 12, 'и'], [0.049414158, 14, 'быть'], [0.029611167, 18, 'этого']]


In [408]:
def sort_by_distance(words, word):
    result = [[calc_single_embedding_dist(word, words[i]), i, words[i]] for i in range(len(words))]
    result.sort()
    return result

print(texts[0])
sort_by_distance(texts[0].split(), "смерть")

он скоро сдохнет и все вернется


[[0.35740493386983874, 0, 'он'],
 [0.3788325250148773, 2, 'сдохнет'],
 [0.4097006469964981, 5, 'вернется'],
 [0.4247136682271958, 3, 'и'],
 [0.43659923374652865, 1, 'скоро'],
 [0.4378321446478367, 4, 'все']]

In [ ]:
def random_letter(word):
    result_word = list(word)
    rand_letter = np.random.choice(list('абвгдежзийклмнопрстуфхцчшщъыьэюя'))
    rand_idx = np.random.randint(0, len(result_word))
    result_word[rand_idx] = rand_letter
    result_word = ''.join(result_word)
    return result_word

In [420]:

def detox_dumb(i, line):
    words = normalize(line).split()
    sorted_words = sort_by_toxicity(words)#sort_by_distance(words, "спасибо")
    
    score = new_checker([texts[i]], [' '.join(words)])
    for toxic_toxicity, toxic_idx, toxic_word in sorted_words:
        

        cur_word = toxic_word
        
     
        candidates = ["спасибо", "любовь", "textbook"]
        candidates += get_top_n(toxic_word, 5)
        candidates = set(candidates)

        
        for candidate in candidates:
            words[toxic_idx] = candidate
            new_score = new_checker([texts[i]], [' '.join(words)])

            if new_score >= score:
                #print(score, "->", new_score, candidate)
                score = new_score
                cur_word = candidate
                
            else:
                words[toxic_idx] = cur_word


    cur_scores.append(score[0])
    return ' '.join(words)


In [ ]:
Original = []
Detoxed = []
for i in tqdm(range(0, len(texts), 50)):
    line = texts[i]
    detoxed = detox_dumb(i, line)

    Original.append(line)
    Detoxed.append(detoxed)


new_checker(Original, Detoxed, show_res = True)

In [ ]:
cur_scores = []
fixed_texts = []
for i in tqdm(range(len(texts))):
    fixed_texts.append(detox_dumb(i, texts[i]))
    print(sum(cur_scores)/len(cur_scores), f"{i+1}/2500")


    #save_comments(fixed_texts, "brand_new_brute")
    

new_checker(texts, fixed_texts, show_res = True)

  0%|          | 0/2500 [00:00<?, ?it/s]

0.6617002717047347 1/2500
0.4910524486390387 2/2500
0.5220229883607312 3/2500
0.536451966308975 4/2500
0.5130364694542039 5/2500
0.49630854306938027 6/2500
0.48921750260979174 7/2500
0.5045397107557006 8/2500
0.5035525196765023 9/2500
0.5077108442177047 10/2500
0.5330090305491023 11/2500
0.5481090008158932 12/2500
0.5820625265660836 13/2500
0.5790872884409836 14/2500
0.5907839103200565 15/2500
0.5857798166427634 16/2500
0.5733982971819436 17/2500
0.5585989591036915 18/2500
0.5694960645200376 19/2500
0.5683389409516001 20/2500
0.555730592112062 21/2500
0.5525973068197785 22/2500
0.5314416323453148 23/2500
0.5469399059785903 24/2500
0.5601116897881422 25/2500
0.5680397520273357 26/2500
0.5670639682935833 27/2500
0.5652103792312054 28/2500
0.5586882249979603 29/2500
0.5668503190993747 30/2500
0.5677235351036524 31/2500
0.569133391901951 32/2500
0.5637623746787689 33/2500
0.5552518379993798 34/2500
0.5571396952542234 35/2500
0.5632988954575926 36/2500
0.5585150659395214 37/2500
0.562996992

#Merges solutions into ultimate one:

In [ ]:
ultimate_solution = []

names = os.listdir(path="/content/drive/MyDrive/YandexCup/submits_pro")
if '.ipynb_checkpoints' in names:
    names.remove('.ipynb_checkpoints')


solutions = {name : [] for name in names}
count = {name : 0 for name in names}
#reading submits
for name in names:
    sol = []
    with open(DRIVE_PATH + f'submits_pro/{name}', 'rt') as f:
        for line in f:
            sol.append(normalize(line))

    solutions[name] = sol[:]
    print(name, len(sol))

for i in tqdm(range(len(texts))):
    scores = {name : 0 for name in names}
    for name in names:
        scores[name] = new_checker([texts[i]], [solutions[name][i]])

    best_sol_name = max(scores, key=scores.get)
    count[best_sol_name] += 1
    ultimate_solution.append(solutions[best_sol_name][i])


print(count)
print(new_checker(texts, ultimate_solution, show_res = True))

In [ ]:
save_comments(ultimate_solution, "BEBRA")

#SUPER BRUTE FORCE:

In [ ]:
solution = []

with open(DRIVE_PATH + 'submits_pro/BEBRA.txt', 'rt') as f:
    for line in f:
        solution.append(normalize(line))

In [ ]:
result = new_checker(texts, solution, return_result = True)
print(result)
print(100*np.mean(result["scores"]))

In [ ]:
sorted_by_score = [[score, i, solution[i]] for i, score in enumerate(result["scores"])]
#sorted_by_score = [[toxicity, i, solution[i]] for i, toxicity in enumerate(result["toxicities"])]
sorted_by_score.sort(reverse=False)

skipped = 0
for score, i, comment in tqdm(sorted_by_score[100:400]):
    print(i, score, comment)
    if len(comment.split()) <= 80:
        solution[i] = detox_letter(i, comment)
    else:
        skipped += 1

In [ ]:
save_comments(solution, "BEBRA_brute")